`IEnumerable<TSource>`→`scalar`  
  
| Method    | Description | SQL equivalents |
| -------- | ------- | ------- |
|`Count`, <br>`LongCount` |Returns the number of elements in the input sequence,<br> optionally satisfying a predicate |COUNT (...)
|`Min`, `Max`|Returns the smallest or largest element in the sequence|MIN (...), <br> MAX (...)
|`Sum`, `Average`|Calculates a numeric sum or average over elements in the <br> sequence| SUM (...), <br> AVG (...)
|`Aggregate`| Performs a custom aggregation | Exception thrown

### Count and LongCount

|Argument | Type
| -----| ----
|***Source sequence*** |`IEnumerable<TSource>`
|***Predicate*** (optional) |`TSource => bool`

In [ ]:
//Count simply enumerates over a sequence, returning the number of items:
int fullCount = new int[] { 5, 6, 7 }.Count(); // 3

The `internal implementation` of ***Enumerable.Count*** tests the input sequence to see whether it happens to `implement ICollection<T>`. If it does, it simply calls `ICollection<T>.Count`; otherwise, it `enumerates over every item`, `incrementing a counter`

In [ ]:
//You can optionally supply a predicate:

int digitCount = "pa55w0rd".Count (c => char.IsDigit (c)); // 3

***LongCount*** does the `same job` as ***Count*** but returns a `64-bit integer`, allowing for sequences of `greater than` ***two billion elements***.

### Min and Max

|Argument | Type
| -----| ----
|***Source sequence*** |`IEnumerable<TSource>`
|***Result selector*** (optional) |`TSource => TResult`

In [ ]:
//Min and Max return the smallest or largest element from a sequence:

int[] numbers = { 28, 32, 14 };
int smallest = numbers.Min(); // 14;
int largest = numbers.Max(); // 32;

In [ ]:
int[] numbers = { 28, 32, 14 };

//If you include a selector expression, each element is first projected:
int smallest = numbers.Max (n => n % 10); // 8;

A ***selector expression*** is `mandatory(اجباری)` if the items themselves are not intrinsically `comparable`—in other words, if they do not implement `IComparable<T>`

In [ ]:
Purchase runtimeError = dbContext.Purchases.Min (); // Error
decimal? lowestPrice = dbContext.Purchases.Min (p => p.Price); // OK

A ***selector expression*** determines `not only` how elements are `compared`, but also the `final result`.

In [ ]:
//To get the cheapest purchase, you need a subquery
Purchase cheapest = dbContext.Purchases
.Where (p => p.Price == dbContext.Purchases.Min (p2 => p2.Price))
.FirstOrDefault();

//best paractice is use OrderBy for this solution

### Sum and Average

|Argument | Type
| -----| ----
|***Source sequence*** |`IEnumerable<TSource>`
|***Result selector*** (optional) |`TSource => TResult`

In [ ]:
decimal[] numbers = { 3, 4, 8 };
decimal sumTotal = numbers.Sum(); // 15
decimal average = numbers.Average(); // 5 (mean value)

In [ ]:
int combinedLength = names.Sum (s => s.Length); // 19

Average Result Types  
  
|Selector type |Result type
|---- | ----
|**decimal** |`decimal`
|**float** |`float`
|**int**, **long**, **double** |`double`

In [ ]:
//This means that the following does not compile (“cannot convert double to int”):
int avg = new int[] { 3, 4 }.Average();

//But this will compile:
double avg1 = new int[] { 3, 4 }.Average(); // 3.5


### Aggregate

***Aggregate*** allows you to specify a `custom accumulation(جمع آوری) algorithm` for implementing `unusual aggregations`.

In [ ]:
public static TSource Aggregate<TSource>(
    this IEnumerable<TSource> source,
    Func<TSource, TSource, TSource> func
);

public static TAccumulate Aggregate<TSource, TAccumulate>(
    this IEnumerable<TSource> source,
    TAccumulate seed,
    Func<TAccumulate, TSource, TAccumulate> func
);

public static TResult Aggregate<TSource, TAccumulate, TResult>(
    this IEnumerable<TSource> source,
    TAccumulate seed,
    Func<TAccumulate, TSource, TAccumulate> func,
    Func<TAccumulate, TResult> resultSelector
);


In [2]:
var numbers = new[] { 1, 2, 3, 4, 5 };
int sum = numbers.Aggregate (10,(total, n) => total + n); // 6
//0+1+2+3
Console.WriteLine(sum);

//The first argument to Aggregate is the seed,from which accumulation starts
// اولین مقدار را صفر در نظر میگیرد، و دومین مقدار 1 میشود

25


<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<p>تابع <strong><code>Aggregate</code></strong> به صورت مکرر عناصر دنباله را به تابعی که تعریف کرده‌اید ارسال می‌کند تا یک مقدار نهایی تولید کند.</p>
</div>

In [4]:
var numbers = new List<int> { 2, 3, 4 };

int result = numbers.Aggregate(2,(total, next) => total * next);

Console.WriteLine(result); // خروجی: 24 (2 * 3 * 4)

48


<div dir="rtl" style="margin:auto; width:90%; font-family:vazirmatn">
<ul><li>مقدار اولیه (seed) تعریف نشده است و اولین عنصر دنباله به عنوان مقدار اولیه در نظر گرفته می‌شود.</li><li>هر عنصر با عنصر قبلی ترکیب می‌شود.</li></ul>
</div>

In [ ]:
// استفاده از مقدار اولیه (Seed)

var numbers = new List<int> { 2, 3, 4 };

int result = numbers.Aggregate( (total, next) => total * next);

Console.WriteLine(result); // خروجی: 240 (10 * 2 * 3 * 4)


In [ ]:
// ایجاد یک رشته از دنباله‌ای از رشته‌ها

var words = new List<string> { "Hello", "World", "LINQ" };

string sentence = words.Aggregate((current, next) => current + " " + next);

Console.WriteLine(sentence); // خروجی: "Hello World LINQ"


In [5]:
// تغییر نوع نتیجه با resultSelector

var numbers = new List<int> { 2, 3, 4 };

string result = numbers.Aggregate(
    "Result:",
    (current, next) => current + " " + next,
    final => final.ToUpper()
);

Console.WriteLine(result); // خروجی: "RESULT: 2 3 4"


RESULT: 2 3 4
